# 02. Conventional RAG based on LangChain

Q: 논문의 내용을 전부 프롬프트에 넣어서 질문하면 대답을 잘할까?

In [1]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

In [2]:
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
from langchain_teddynote import logging

logging.langsmith(project_name="Paper-Agent")

LangSmith 추적을 시작합니다.
[프로젝트명]
Paper-Agent


In [4]:
loader = PyMuPDFLoader("../data/Attention Is All You Need.pdf")
docs = loader.load()
print(f"문서의 페이지수: {len(docs)}")

문서의 페이지수: 15


In [11]:
paper_contents = ""

for i in range(len(docs)):
    paper_contents += docs[i].page_content

print(len(paper_contents))
print(paper_contents[:300])

39483
Provided proper attribution is provided, Google hereby grants permission to
reproduce the tables and figures in this paper solely for use in journalistic or
scholarly works.
Attention Is All You Need
Ashish Vaswani∗
Google Brain
avaswani@google.com
Noam Shazeer∗
Google Brain
noam@google.com
Niki Par


In [25]:
# 단계 6: 프롬프트 생성(Create Prompt)
prompt = PromptTemplate.from_template(
    """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the question. 
If you don't know the answer, just say that you don't know. 
Answer in Korean.

#Context: 
{context}

#Question:
{question}

#Answer:"""
)

In [26]:
llm = ChatOpenAI(model_name="gpt-4o", temperature=0)

In [28]:
chain = (
    {"context": lambda x: paper_contents, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [29]:
chain

{
  context: RunnableLambda(lambda x: paper_contents),
  question: RunnablePassthrough()
}
| PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. \nUse the following pieces of retrieved context to answer the question. \nIf you don't know the answer, just say that you don't know. \nAnswer in Korean.\n\n#Context: \n{context}\n\n#Question:\n{question}\n\n#Answer:")
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x116ef3990>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x116ef3750>, root_client=<openai.OpenAI object at 0x116eaa710>, root_async_client=<openai.AsyncOpenAI object at 0x116ef1190>, model_name='gpt-4o', temperature=0.0, model_kwargs={}, openai_api_key=SecretStr('**********'))
| StrOutputParser()

In [30]:
# 체인 실행(Run Chain)
question = "이 논문의 제목 알려줘"
response = chain.invoke(question)
print(response)

"Attention Is All You Need"입니다.


In [31]:
# 체인 실행(Run Chain)
question = "이 논문의 주제가 무엇이야?"
response = chain.invoke(question)
print(response)

이 논문의 주제는 "Transformer"라는 새로운 네트워크 아키텍처에 관한 것입니다. Transformer는 주로 주의 메커니즘에 기반하여 순환 및 합성곱을 완전히 배제하고 입력과 출력 간의 전역적인 의존성을 모델링합니다. 이 모델은 기계 번역과 같은 시퀀스 변환 작업에서 뛰어난 성능을 보이며, 더 병렬화가 가능하고 훈련 시간이 크게 단축된다는 장점을 가지고 있습니다.


In [23]:
# 체인 실행(Run Chain)
question = "이 논문의 저자가 누구야?"
response = chain.invoke(question)
print(response)

이 논문의 저자는 Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Łukasz Kaiser, Illia Polosukhin입니다.


In [24]:
# 체인 실행(Run Chain)
question = "이 논문의 Abstract 설명해줘."
response = chain.invoke(question)
print(response)

이 논문의 Abstract에서는 기존의 순차 변환 모델들이 복잡한 순환 신경망(RNN)이나 합성곱 신경망(CNN)을 기반으로 하고 있으며, 이러한 모델들은 인코더와 디코더를 주의 메커니즘을 통해 연결한다고 설명합니다. 그러나 이 논문에서는 주의 메커니즘만을 기반으로 하는 새로운 간단한 네트워크 아키텍처인 Transformer를 제안합니다. Transformer는 순환과 합성곱을 완전히 배제하고, 주의 메커니즘만을 사용하여 입력과 출력 간의 전역적인 의존성을 모델링합니다. 실험 결과, Transformer는 기계 번역 작업에서 더 높은 품질을 보이며, 병렬화가 더 용이하고 훈련 시간이 크게 단축된다는 것을 보여줍니다. WMT 2014 영어-독일어 번역 작업에서 28.4 BLEU 점수를 기록하며 기존의 최고 성능을 2 BLEU 이상 초과하였고, 영어-프랑스어 번역 작업에서도 새로운 단일 모델 최고 성능인 41.8 BLEU 점수를 기록했습니다. Transformer는 대규모 및 제한된 훈련 데이터를 사용한 영어 구문 분석 작업에서도 성공적으로 적용되어 다른 작업에도 잘 일반화된다는 것을 보여줍니다.


## Result

- 논문 전체 내용을 넣어버리면 원하는 결과가 나오기는 함
- 하지만 토큰 사용량이 많아짐(RAG 사용시 약 1,300 --> 전체 텍스트 사용시 약 10,400)